# Dataset: All Conference
#### Sources: Wikipedia, Manual Data Aggregation (CSV)
> The All Conference dataset works a bit differently.  Half of the data comes from Wikipedia, but Wiki doesn't have standard representations of all the conferences.  Instead, work was done to manually grab that data from multiple disparate sources and normalized into a single CSV file.  This notebook will step through getting data from both sources, normalizing and integrating the findings before summarizing for the final linking step.

In [1]:
#hide
import core_constants as cc
import functions as fx
import requests
import lxml
import time
import json
import pandas as pd
import sqlite3 as sql

## Set Notebook Settings

In [3]:
#Load the page titles you are interested in
pageList = [['sec', 'All-SEC_football_team'], ['bigtwelve', 'All-Big_12_Conference_football_team'], ['bigten', 'All-Big_Ten_Conference_football_team']]
years = cc.get_defYears()
headers= cc.get_header()
csvFile = "..//scrapedData//allConf.csv"

## Get & Save the Wikipedia HTML
#### Source: https://en.wikipedia.org/wiki/2020_Big_Ten_Conference_football_season#All-Conference_Teams
> These Wiki pages contain the all conference records.  We cycle through by conference - which for these three conferences follow the same page layout and url schema for each year we care about.  

##### To run - convert the below cell back to code

In [ ]:
fx.get_WikipediaAllConf(pageList, headers, years)

### Process & Save Big Ten/Big 12 Data from Wikipedia
> These files are already saved locally.  Saves it out to a json file

In [ ]:
teamDir = ['..//html//wikipedia//allconference//bigten//', '..//html//wikipedia//allconference//bigtwelve//']

with open("..//scrapedData//Ten12AllConf.json", "w") as write_file:
    json.dump(fx.process_WikipediaBigTenBigTwelve(teamDir), write_file)

### Process & Save SEC Data from Wikipedia
> These files are already saved locally.  Saves it out to a json file

In [ ]:
teamDir = '..//html//wikipedia//allconference//sec//'

with open("..//scrapedData//SECAllConf.json", "w") as write_file:
    json.dump(fx.process_WikipediaSEC(teamDir), write_file)

## Get & Process the CSV File for the other Conferences

> The CSV file contains the all conference records for the Pac12, ACC and the group of 5 conferences

##### Input: //scrapedData/allconf.csv
##### Output: //scrapedData/allConf.json

In [4]:
with open("..//scrapedData//allConf.json", "w") as write_file:
    write_file.write(json.dumps(fx.process_csvAllConf(fx.get_csvAllConf(csvFile))))

## Save to DB

In [5]:
fx.toDB_AllConference()

[{'ID': 'drewlock_missouri', 'AllConferenceTeam': '1', 'PlayerName': 'drewlock', 'College': 'missouri'}, {'ID': 'jarrettstidham_auburn', 'AllConferenceTeam': '1', 'PlayerName': 'jarrettstidham', 'College': 'auburn'}, {'ID': 'kerryonjohnson_auburn', 'AllConferenceTeam': '1', 'PlayerName': 'kerryonjohnson', 'College': 'auburn'}, {'ID': 'bennysnell_kentucky', 'AllConferenceTeam': '2', 'PlayerName': 'bennysnell', 'College': 'kentucky'}, {'ID': 'nickchubb_georgia', 'AllConferenceTeam': '2', 'PlayerName': 'nickchubb', 'College': 'georgia'}, {'ID': 'derriusguice_lsu', 'AllConferenceTeam': '2', 'PlayerName': 'derriusguice', 'College': 'lsu'}, {'ID': 'ajbrown_olemiss', 'AllConferenceTeam': '1', 'PlayerName': 'ajbrown', 'College': 'olemiss'}, {'ID': 'calvinridley_alabama', 'AllConferenceTeam': '1', 'PlayerName': 'calvinridley', 'College': 'alabama'}, {'ID': 'jmonmoore_missouri', 'AllConferenceTeam': '2', 'PlayerName': 'jmonmoore', 'College': 'missouri'}, {'ID': 'christiankirk_texasam', 'AllConfe

'DB Write is done'